# Telco Customer Churn: Model Deployment

## 1. Setup

In [ ]:
import pandas as pd
import joblib
import sys
import os
import yaml

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.model_persistence import ModelPersistence
from src.inference_engine import InferenceEngine
from src.data_loader import TelcoDataLoader

with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print('Setup complete')

## 2. Save Trained Model

In [ ]:
# Load best model from previous training
from src.ensemble_models import XGBoostChurnModel
from src.preprocessor import DataPreprocessor
from sklearn.model_selection import train_test_split

loader = TelcoDataLoader()
df = loader.load_raw_data()
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(subset=['TotalCharges'], inplace=True)

X = df.drop(config['target'], axis=1)
y = (df[config['target']] == 'Yes').astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

preprocessor = DataPreprocessor().create_preprocessing_pipeline()
X_train_transformed = preprocessor.fit_transform(X_train)

# Train model
xgb_params = config['models']['xgboost']
model = XGBoostChurnModel(**xgb_params)
model.train(X_train_transformed, y_train)

# Save model and preprocessor
os.makedirs('../models', exist_ok=True)
joblib.dump(model.model, '../models/churn_model.pkl')
joblib.dump(preprocessor, '../models/preprocessor.pkl')

print('Model and preprocessor saved to ../models/')

## 3. Load Model for Inference

In [ ]:
loaded_model = joblib.load('../models/churn_model.pkl')
loaded_preprocessor = joblib.load('../models/preprocessor.pkl')
print('Model loaded successfully')

## 4. Make Predictions on New Data

In [ ]:
# Sample new customer data
new_customer = pd.DataFrame([{
    'customerID': 'NEW001',
    'gender': 'Male',
    'SeniorCitizen': 0,
    'Partner': 'Yes',
    'Dependents': 'No',
    'tenure': 12,
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'Fiber optic',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'No',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'Yes',
    'StreamingMovies': 'Yes',
    'Contract': 'Month-to-month',
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges': 89.95,
    'TotalCharges': 1079.40
}])

# Preprocess and predict
new_customer_processed = loaded_preprocessor.transform(new_customer.drop('customerID', axis=1))
churn_prediction = loaded_model.predict(new_customer_processed)
churn_probability = loaded_model.predict_proba(new_customer_processed)[:, 1]

print(f'Churn Prediction: {"Yes" if churn_prediction[0] == 1 else "No"}')
print(f'Churn Probability: {churn_probability[0]:.2%}')

## 5. Batch Predictions

In [ ]:
# Make predictions on test set
X_test_processed = loaded_preprocessor.transform(X_test)
batch_predictions = loaded_model.predict(X_test_processed)
batch_probabilities = loaded_model.predict_proba(X_test_processed)[:, 1]

# Create predictions dataframe
predictions_df = pd.DataFrame({
    'prediction': batch_predictions,
    'churn_probability': batch_probabilities
}, index=X_test.index)

# Save predictions
os.makedirs('../data/predictions', exist_ok=True)
predictions_df.to_csv('../data/predictions/churn_predictions.csv')
print(f'Batch predictions saved: {predictions_df.shape[0]} records')

## Summary

Model deployment workflow:
1. Trained and saved model with preprocessor
2. Loaded model for inference
3. Made single prediction on new customer
4. Generated batch predictions
5. Saved predictions to file